In [2]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, LSTM, Dropout
from keras.models import Sequential
from keras import regularizers
from keras.callbacks import EarlyStopping
import random
import warnings

warnings.filterwarnings(action='ignore')

In [14]:
# 데이터 불러오기
df1 = pd.read_csv('../data/review-rating1.csv')
df2 = pd.read_csv('../data/review-rating2.csv')
df3 = pd.read_csv('../data/review-rating3.csv')
df4 = pd.read_csv('../data/review-rating4.csv')
df = pd.concat([df1, df2, df3, df4])

# 중복값 제거
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

# 랜덤으로 200개의 리뷰 가져오기
random_indices = random.sample(range(len(df)), k=200)
df_random = df.loc[random_indices]
df_random = df_random[['영화ID', '평균평점']]
df_random

In [4]:
movie = pd.read_csv('../data/movie_info.csv')
movie = movie[['영화ID', '줄거리']]
movie

,영화ID,줄거리
0,0,"가리봉동 소탕작전 후 4년 뒤, 금천서 강력반은 베트남으로 도주한 용의자를 인도받아..."
1,1,<아바타: 물의 길>은 판도라 행성에서 '제이크 설리'와 '네이티리'가 이룬 가족이...
2,2,최고의 파일럿이자 전설적인 인물 매버릭(톰 크루즈)은 자신이 졸업한 훈련학교 교관으...
3,3,‘미스테리오’의 계략으로 세상에 정체가 탄로난 스파이더맨 ‘피터 파커’는 하루 아침...
4,4,"1592년 4월, 조선은 임진왜란 발발 후 단 15일 만에 왜군에 한양을 빼앗기며 ..."
...,...,...
8925,11278,"스타쓰는 우즈베키스탄 타슈켄트에 살고 있는 한국계 청년으로, 과거 스탈린 시절 연해..."
8926,11279,"세 명의 인물이 등장한다. 트랜스젠더 장지영, 그녀에 의해 필리핀에서 입양된 로이탄..."
8927,11280,과학자 ‘윌 로드만(제임스 프랭코 분)’은 알츠하이머 병에 걸린 아버지(존 리스고 ...
8928,11281,외딴 산골에서 할아버지에게 사냥을 배워 솜씨 좋은 사냥꾼으로 성장하고 있는 소녀 하...


In [5]:
summary = pd.merge(df_random, movie, on='영화ID')
summary.rename({'줄거리':'summary', '평균평점':'rating'}, axis=1, inplace=True)
summary.dropna(inplace=True)
summary.reset_index(inplace=True, drop=True)
summary

,영화ID,rating,summary
0,172,9.40,해리 포터(다니엘 래드클리프 분)는 위압적인 버논 숙부(리챠드 그리피스 분)와 냉담...
1,6096,4.38,1945년 태평양전쟁에서 패색이 짙어진 일본은 연합군으로부터 포츠담선언인 무조건 항...
2,3081,8.69,"비밀 임무를 맡게 된 경찰 아걸(장국영), 동생인 아걸을 위해 사건에 뛰어들게 되는..."
3,2612,5.62,마을의 폐가에서 발견한 책 한 권. 책을 펼치는 순간 멈출 수 없는 공포스러운 스토...
4,3421,8.56,바이올린의 장인 부조티(Nicolo Bussotti: 카를로 세치 분)는 바이올린 ...
...,...,...,...
193,6933,7.81,한 병원의 간호조무사 해림(서영희)과 의사 혁규(변요한)는 심장 이식이 필요한 전신...
194,6142,8.22,"암투병 중이던 엄마가 돌아가신 지 1년, 아버지가 내내 어렵고 불편한 한없이 평범하..."
195,4647,6.89,"아무도 몰랐던 형제, 흥부 - 놀부 양반들의 권력 다툼으로 백성들의 삶이 날로 피폐..."
196,4736,8.65,온 세상을 차가운 크리스탈로 바꾸려하는 ‘크리스탈 마스터’로부터 마법을 지켜야하는 ...


In [6]:
for i, rating in enumerate(summary['rating']):
    if rating > summary.rating.mean():
        summary.loc[i, 'rating'] = 1
    else:
        summary.loc[i, 'rating'] = 0

In [7]:
summary

,영화ID,rating,summary
0,172,1.0,해리 포터(다니엘 래드클리프 분)는 위압적인 버논 숙부(리챠드 그리피스 분)와 냉담...
1,6096,0.0,1945년 태평양전쟁에서 패색이 짙어진 일본은 연합군으로부터 포츠담선언인 무조건 항...
2,3081,1.0,"비밀 임무를 맡게 된 경찰 아걸(장국영), 동생인 아걸을 위해 사건에 뛰어들게 되는..."
3,2612,0.0,마을의 폐가에서 발견한 책 한 권. 책을 펼치는 순간 멈출 수 없는 공포스러운 스토...
4,3421,1.0,바이올린의 장인 부조티(Nicolo Bussotti: 카를로 세치 분)는 바이올린 ...
...,...,...,...
193,6933,1.0,한 병원의 간호조무사 해림(서영희)과 의사 혁규(변요한)는 심장 이식이 필요한 전신...
194,6142,1.0,"암투병 중이던 엄마가 돌아가신 지 1년, 아버지가 내내 어렵고 불편한 한없이 평범하..."
195,4647,1.0,"아무도 몰랐던 형제, 흥부 - 놀부 양반들의 권력 다툼으로 백성들의 삶이 날로 피폐..."
196,4736,1.0,온 세상을 차가운 크리스탈로 바꾸려하는 ‘크리스탈 마스터’로부터 마법을 지켜야하는 ...


In [8]:
X = summary['summary']
y = summary['rating']

In [9]:
# 특수문자,기호 제거
X = X.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]", "", regex=True)
# 공백 제거
X = X.replace('^ +', "", regex=True)
X.replace('', np.nan, inplace=True)

# 불용어 사전
with open("c:/data/movie_list/불용어사전.txt") as f:
    lines = f.read().splitlines()
stopwords = []
for line in lines:
    stopwords.append(line.split("\t")[0])

# 형태소 분석
okt = Okt()
X_lis = []
if isinstance(X, str):
    X = [X]  # 문자열을 리스트로 변환
for i, sentence in enumerate(X):
    try:
        temp_X = okt.morphs(sentence, stem=True) # 토큰화
    except:
        print(i, sentence)
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_lis.append(temp_X)

# 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_lis)

# 출현빈도가 3회 미만인 단어들
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어수
rare_cnt = 0
total_freq = 0
rare_freq = 0
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

# 단어 집합의 크기
vocab_size = total_cnt - rare_cnt + 1

# 텍스트를 숫자 시퀀스로 변환
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_lis) 
X = tokenizer.texts_to_sequences(X_lis)

# 줄거리의 최대 길이
max_len = max(len(l) for l in X)

# 독립변수 패딩
X = pad_sequences(X, maxlen=max_len)

In [10]:
# 종속변수를 array로 변환
y = np.array(y)

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

# 모델 생성하는 함수
def create_model(embedding_dim=250, units=256, dense1_units=128, dense2_units=64):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(LSTM(units=units))
    model.add(Dropout(0.2))
    # model.add(Dense(units=dense1_units, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    # model.add(Dropout(0.3))
    # model.add(Dense(units=dense2_units, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    # model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 모델 생성
model = KerasRegressor(build_fn=create_model, verbose=0)

# 하이퍼파라미터 탐색 영역을 정의
param_grid={
    'epochs': [10],
    'batch_size': [16, 24, 32],
    'embedding_dim': list(range(10, 50, 10)),
    'units': list(range(4, 16, 4)),
    # 'dense1_units': [64, 128],
    # 'dense2_units': [32, 64],
}

# 파라미터 검색
grid=GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3)

# 모델 학습
es = EarlyStopping(monitor='val_accuracy', mode='min', patience=3)
grid_result = grid.fit(X, y, validation_split=0.2, callbacks=es)

# 결과 출력
print(grid_result.best_params_)
print(grid_result.best_score_)

In [11]:
# 모델 생성
model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(LSTM(16))
model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(Dropout(0.3))
# model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          73250     
                                                                 
 lstm (LSTM)                 (None, 16)                4288      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 77,555
Trainable params: 77,555
Non-trainable params: 0
_________________________________________________________________


In [12]:
# 모델 학습
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=3)
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2, callbacks=es)

Epoch 1/10
5/5 [==============================] - 13s 538ms/step - loss: 0.6711 - accuracy: 0.7532 - val_loss: 0.6156 - val_accuracy: 1.0000
Epoch 2/10
5/5 [==============================] - 0s 24ms/step - loss: 0.5749 - accuracy: 0.9114 - val_loss: 0.4192 - val_accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 0s 20ms/step - loss: 0.3731 - accuracy: 0.9114 - val_loss: 0.1964 - val_accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 0s 21ms/step - loss: 0.3069 - accuracy: 0.9114 - val_loss: 0.1357 - val_accuracy: 1.0000


In [13]:
# 결과 시리즈로 저장
series = model.predict(X)
series

7/7 [==============================] - 0s 4ms/step


array([[0.8831784 ],
       [0.8460717 ],
       [0.8792651 ],
       [0.85273004],
       [0.87952787],
       [0.8849253 ],
       [0.8605877 ],
       [0.8840285 ],
       [0.84344983],
       [0.88174427],
       [0.8802254 ],
       [0.87713844],
       [0.8768688 ],
       [0.8585834 ],
       [0.8838665 ],
       [0.8786571 ],
       [0.84240556],
       [0.8813774 ],
       [0.8588888 ],
       [0.8809996 ],
       [0.8412411 ],
       [0.88420683],
       [0.87244785],
       [0.87917227],
       [0.8812667 ],
       [0.87872064],
       [0.87872064],
       [0.8709975 ],
       [0.84308374],
       [0.8790294 ],
       [0.8749554 ],
       [0.88161784],
       [0.8819549 ],
       [0.87918   ],
       [0.8824164 ],
       [0.8582557 ],
       [0.85970783],
       [0.8785429 ],
       [0.8700335 ],
       [0.8767594 ],
       [0.8812441 ],
       [0.8732181 ],
       [0.87723786],
       [0.879324  ],
       [0.8718622 ],
       [0.8683194 ],
       [0.87651145],
       [0.875